In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import glob
from pathlib import Path

In [1]:
df = pd.DataFrame(columns=['Title_all', 'Title', 'Season', 'Episode', 'Image', 'Description', 'Genre', 'Duration', 'Date'])

Genre = 'food'

files = glob.glob(f'./data2/BBC/{Genre}/*.html')


for file in files:
    try:
        with open(file) as fp:
            soup = BeautifulSoup(fp, 'html.parser')
        test_site_name = soup.find('meta', attrs={'property': 'og:site_name'})
        test_title = soup.find('meta', attrs={'property': 'og:title'})
        test_image = soup.find('meta',  attrs={'property': 'og:image'})
        test_description = soup.find('meta',  attrs={'property': 'og:description'})
        meta_test = soup.findAll('div', attrs={'id': 'orb-modules'})

        if (test_title != None) and (test_image != None) and (test_description != None) and (meta_test != None):
        # extract the data from each file
        #extract title
            Title_all = test_title['content']
        #Extract Image
            Image = test_image['content']
        #Extract Description
            Description = test_description['content']

            if test_site_name['content'] == 'BBC iPlayer':
                for tag in meta_test:
                    tdTags = tag.find_all("span", {"class": "episode-metadata__text"})
                    tdTags2 = tag.find_all("span", {"class": "typo typo--bold play-cta__title typo--buzzard"})
                    tdTags3 = tag.find_all("span", {"class": "typo typo--skylark play-cta__subtitle"})

                    List = []
                    Titles = []

                    for items in tdTags:
                        List.append(items.text)
                    for i in tdTags2:
                        Titles.append(i.text)
                    for d in tdTags3:
                        Titles.append(d.text)

                    Title = Titles[0]
                    if tdTags3 ==  []:
                        Season = ""
                    else:
                        Season = Titles[1]

                    Duration = List[0]
                    Date = List[1]
                    Episode = ""

            elif test_site_name['content'] == 'BBC':
                for tag in meta_test:
                    tdTags = tag.find_all("div", {"class": "map__intro"})
                    tdTags2 = tag.find_all("a", {"class": "context__item"})
                    tdTags3 = tag.find_all("h1", {"class": "no-margin"})
                    List = []
                    for items in tdTags:
                        List.append(items.text)
                    for i in tdTags2:
                        List.append(i.text)
                    for d in tdTags3:
                        List.append(d.text)

                Duration = List[0]
                Title = List[1]
                Season = List[2]
                if len(List) == 4:
                    Episode = List[3]
                else:
                    Episode = ""
                Date = ""



            df = df.append({'Title_all': Title_all, 'Title': Title, 'Season': Season, 'Episode': Episode, 'Image': Image, 'Description': Description, 'Genre': Genre, 'Duration': Duration, 'Date': Date}, ignore_index=True)
        else:
            pass
    except:
        pass

In [22]:
df.to_csv(f'../data/websraped_raw.csv', index=False, sep=";")